# Ebay (Germany) Car Sales Analysis

We will analysing a [dataset](https://www.kaggle.com/orgesleka/used-cars-database/data) of used cars from Ebay. 

In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding="Latin-1")

Let's do a quick analysis of the dataframe. From below we can see that some data wrangling. For instance lastSeen is an object(string) however it should be datatime64. Also, some colomns do have null values.

In [2]:
autos.info()
autos.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In order to remove incosistences we will rewrite headers for the table and change camelCase to snake_case.

In [3]:
auto_columns = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'num_photos', 'postal_code',
       'last_seen']
autos.columns = auto_columns

Let's analyse the data set and look for potential columns which might be excluded or changed.

Obeservations:
* num_photos has only one value and can be dropped
* Seller & offer_type colomns looks odd and need to be investigated
* Many columns format need to be changed 

In [4]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-04-04 16:40:33,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [5]:
autos["offer_type"].value_counts()

Angebot    49999
Gesuch         1
Name: offer_type, dtype: int64

In [6]:
autos["seller"].value_counts()

privat        49999
gewerblich        1
Name: seller, dtype: int64

Clearly "offer_type" & "seller" columns can be removed. We also will clear price & odometer columns

In [7]:
autos["price"] = autos["price"].str.replace("$","").str.replace(",","").astype(int)
autos["odometer"] = autos["odometer"].str.replace("km","").str.replace(",","").astype(int)

In [8]:
autos.rename({"price":"price_usd", "odometer":"odometer_km"},axis=1,inplace=True)

We will do a futher exploration of the data set

In [9]:
print("count unique values in odometer",autos["odometer_km"].unique().shape)
autos["odometer_km"].describe()

count unique values in odometer (13,)


count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [10]:
print("count unique values in price",autos["price_usd"].unique().shape)
autos["price_usd"].value_counts().head(5).sort_index(ascending=True)

count unique values in price (2357,)


0       1421
500      781
1000     639
1500     734
2500     643
Name: price_usd, dtype: int64

In [11]:
print("Max price -", autos["price_usd"].value_counts().sort_index(ascending=False).head(10))

Max price - 99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
Name: price_usd, dtype: int64


Clearly top 10 entries are outliers we will remove them.

In [12]:
autos = autos[autos["price_usd"].between(1,999990)]

In [13]:
autos["date_crawled"].str[:10].value_counts(normalize=True,dropna=False).sort_index()

2016-03-05    0.025326
2016-03-06    0.014043
2016-03-07    0.036013
2016-03-08    0.033295
2016-03-09    0.033089
2016-03-10    0.032183
2016-03-11    0.032574
2016-03-12    0.036919
2016-03-13    0.015669
2016-03-14    0.036548
2016-03-15    0.034283
2016-03-16    0.029609
2016-03-17    0.031627
2016-03-18    0.012910
2016-03-19    0.034777
2016-03-20    0.037887
2016-03-21    0.037372
2016-03-22    0.032986
2016-03-23    0.032224
2016-03-24    0.029342
2016-03-25    0.031606
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034119
2016-03-30    0.033686
2016-03-31    0.031833
2016-04-01    0.033686
2016-04-02    0.035477
2016-04-03    0.038607
2016-04-04    0.036486
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

We can see that data looks like uniform distribution and that the data was gather over ont month period.

In [14]:
autos["registration_year"].describe()

count    48566.000000
mean      2004.755014
std         88.643020
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

From analysing "registration_year" column we can see that there are a few outliers eg. "9999" or "1000". STD of 88 also show us a presence of outliers. We shall keep only values from 1910 to 2016 simply because cars earlier than 1910 are extremely rare and this data set is from 2016.

In [15]:
autos = autos[autos["registration_year"].between(1900,2016)]

In [16]:
autos["registration_year"].value_counts(normalize=True).head(10)

2000    0.067606
2005    0.062894
1999    0.062058
2004    0.057902
2003    0.057817
2006    0.057195
2001    0.056467
2002    0.053254
1998    0.050619
2007    0.048777
Name: registration_year, dtype: float64

After cleaning we can see that most of the cars are from 1999-2006

### Exploring Different Brands

In [17]:
autos["brand"].describe()

count          46682
unique            40
top       volkswagen
freq            9863
Name: brand, dtype: object

In [18]:
autos["brand"].value_counts(normalize=True)

volkswagen        0.211281
bmw               0.110042
opel              0.107579
mercedes_benz     0.096461
audi              0.086564
ford              0.069898
renault           0.047149
peugeot           0.029840
fiat              0.025642
seat              0.018273
skoda             0.016409
nissan            0.015274
mazda             0.015188
smart             0.014160
citroen           0.014010
toyota            0.012703
hyundai           0.010025
sonstige_autos    0.009811
volvo             0.009147
mini              0.008761
mitsubishi        0.008226
honda             0.007840
kia               0.007069
alfa_romeo        0.006641
porsche           0.006127
suzuki            0.005934
chevrolet         0.005698
chrysler          0.003513
dacia             0.002635
daihatsu          0.002506
jeep              0.002271
subaru            0.002142
land_rover        0.002099
saab              0.001649
jaguar            0.001564
daewoo            0.001500
trabant           0.001392
r

From the quck exporation above we can see that german cars are the most popular and we have 40 car manufacturers in total . We will focus on top 5 german barnds and add an obvius outlier "Lada" for the comaprison. 

In [19]:
price_mean_brans = {}
brand_filter = autos["brand"].isin(['bmw','volkswagen','opel','mercedes_benz','audi','lada'])


In [56]:
selected_brands = autos.loc[brand_filter]["brand"].unique()

prices_list = {}

for b in selected_brands:
    brand = autos[(autos["brand"] == b)]
    mean_b = round(brand["price_usd"].mean(),0)
    prices_list[b] = mean_b

print(prices_list)

{'opel': 2975.0, 'bmw': 8333.0, 'volkswagen': 5503.0, 'audi': 9337.0, 'lada': 2688.0, 'mercedes_benz': 8628.0}


Some insights from the analysis of top 5 german brands and a little Russian outlier:

* Audi is by far the most expensive brand with an average price 9337. 
* BMW & Mercedes are slightly cheaper with 8333 & 8628 average prices respectively 
* Volskwagen is storngly in the middle with an average price of 5503 
* Finally, quite surprisingly Opel is just sliglthy more expensive than lada

In [73]:
mileage_list = {}

for b in selected_brands:
    brand = autos[(autos["brand"] == b)]
    mean_b = round(brand["odometer_km"].mean(),0)
    mileage_list[b] = mean_b

milleage_price_df = pd.DataFrame(pd.Series(prices_list).sort_values(ascending=False), columns=['mean_price'])
milleage_price_df['mean_mileage'] = pd.Series(mileage_list).sort_values(ascending=False)

In [74]:
milleage_price_df

,mean_price,mean_mileage
audi,9337.0,129157.0
mercedes_benz,8628.0,130788.0
bmw,8333.0,132573.0
volkswagen,5503.0,128709.0
opel,2975.0,129310.0
lada,2688.0,83519.0


From the above, we can see that pretty much all the cars have an average mileage around 120,000km with an exception of 'BMW' and 'Lada' where mileage is higher for 'BMW' and significantly lower for 'Lada'.

This makes 'Lada' quite an interesting choice since both price & mileage are low. 